In [1]:
import pandas as pd
import numpy as np
import random
import os

# implementing random seed for control
sd = 42
np.random.seed(sd)

# school names for consistency 
schools = [
    # 'Oakland Middle School',
    # 'Siegel Middle School',
    # 'Whitworth-Buchanan Middle School',
    # 'Christiana Middle School',
    # 'Smyrna Middle School',
    # 'Stewarts Creek Middle School',
    # 'Rockvale Middle School',
    # 'Rocky Fork Middle School',
    # 'Blackman Middle School',
    # # 'Thurman Francis Arts Academy',
    # 'Rock Springs Middle School',
    'LaVergne Middle School'
]


### Updating Room Assignment File

In [7]:
# Read in room_assignment doc, , then assign the ", B"'s to extra rooms
ra_file_name = 'room_assignments_by_demand.csv'
ra_p = f'../YouScienceData/Schedules/2023/LaVergne Middle School/{ra_file_name}'
room_assignments_df = pd.read_csv(p).drop("Unnamed: 0", axis=1)

# determine the pathways to be split
rooms_of_split_pathways = [room for room in list(room_assignments_df.Room) if 'x2' in str(room)]

# determine unassigned rooms to be utilized 
rooms_to_be_assigned = [room for room in list(room_assignments_df[room_assignments_df["POS"].isnull()].Room)]

# pathways to be rewritten in all_students.csv
pathways_to_rewrite = []

# append original pathways with ", A"
i = 0
for room in rooms_of_split_pathways:
    # get pathway name
    original_pathway = room_assignments_df.loc[room_assignments_df.Room == room]["POS"].values[0]
    pathways_to_rewrite.append(original_pathway)
    # append existing with ', A'
    room_assignments_df.loc[room_assignments_df.Room == room, "POS"] = original_pathway + ', A'
    # assign to open room with ', B'
    room_assignments_df.loc[room_assignments_df.Room == rooms_to_be_assigned[i], "POS"] = original_pathway + ', B'
    room_assignments_df.loc[room_assignments_df.Room == rooms_to_be_assigned[i], "School"] = schools[0]

    # revert capacity
    temp_cap = room_assignments_df.loc[room_assignments_df.Room == room]["Capacity"]
    extra_room_cap = room_assignments_df.loc[room_assignments_df.Room == rooms_to_be_assigned[i]]["Capacity"].values[0]
    room_assignments_df.loc[room_assignments_df.Room == room, "Capacity"] = temp_cap - extra_room_cap
    # remove x2 tag 
    room_assignments_df.loc[room_assignments_df.Room == room, "Room"] = int(room[:-2])
    
    i += 1


In [8]:
room_assignments_df

,Room,Capacity,POS,School
0,612,25,"Virtual Reality, A",LaVergne Middle School
1,611,25,"Marketing Management, A",LaVergne Middle School
2,610,25,"Health Science, A",LaVergne Middle School
3,609,25,"Audio Visual Production, A",LaVergne Middle School
4,608,25,"Veterinary & Animal Science/Horticulture, A",LaVergne Middle School
5,607,25,"Educational Counseling & Social Services, A",LaVergne Middle School
6,606,25,Criminal Justice & Correction Services,LaVergne Middle School
7,605,25,Cosmetology/Barbering,LaVergne Middle School
8,604,25,Networking Systems,LaVergne Middle School
9,603,25,Leadership in Government,LaVergne Middle School


### Updating all_students.csv File

In [31]:
def export_POS_rosters(pos_keys, cleaned_df):

    for pos_key in pos_keys:
        # fixing unexpected '/' in pos_key str
        new_key = pos_key.replace('/', ', ')
            
        if new_key != pos_key:
            pos_str_exceptions.add(pos_key)
        
        # get block slices, index starting with 1 (teacher-friendly)
        b1_df = cleaned_df.loc[cleaned_df.B1 == pos_key]
        b1_df.index = np.arange(1, len(b1_df) + 1)
        b2_df = cleaned_df.loc[cleaned_df.B2 == pos_key]
        b2_df.index = np.arange(1, len(b2_df) + 1)
        b3_df = cleaned_df.loc[cleaned_df.B3 == pos_key]
        b3_df.index = np.arange(1, len(b3_df) + 1)
        b4_df = cleaned_df.loc[cleaned_df.B4 == pos_key]
        b4_df.index = np.arange(1, len(b4_df) + 1)

        # create .xlsx file for pathway
        with pd.ExcelWriter(f'../YouScienceData/Schedules/{event_year}/{school}/Pathway_Rosters/{new_key}.xlsx') as exwriter:
            b1_df.to_excel(exwriter, sheet_name='Block 1')
            b2_df.to_excel(exwriter, sheet_name='Block 2')
            b3_df.to_excel(exwriter, sheet_name='Block 3')
            b4_df.to_excel(exwriter, sheet_name='Block 4')
    return 

In [9]:
as_file_name = f'{schools[0]}_all_students.csv'
as_p = f'../YouScienceData/Schedules/2023/LaVergne Middle School/{as_file_name}'
all_students_df = pd.read_csv(as_p).drop("Unnamed: 0", axis=1)

In [28]:

for pathway in pathways_to_rewrite:
    # determine how many to move to pathway, A/B
    a_path_cap = room_assignments_df.loc[room_assignments_df.POS == pathway + ', A']["Capacity"].values[0]
    b_path_cap = room_assignments_df.loc[room_assignments_df.POS == pathway + ', B']["Capacity"].values[0]
    
    for block in ['B1','B2','B3','B4']:
        student_emails = list(all_students_df.loc[all_students_df[block] == pathway]["Email"])
        print(f'{pathway} {block} total = {len(student_emails)}')
        while len(student_emails) > a_path_cap:
            # add suffix ", A" to their pathway
            all_students_df.loc[all_students_df.Email == student_emails[0], block] = pathway + ", A"
            student_emails.pop(0) 
        
        while len(student_emails) > 0:
            # add suffix ", B" to their pathway
            all_students_df.loc[all_students_df.Email == student_emails[0], block] = pathway + ", B"
            student_emails.pop(0)
        print(len(student_emails))


Virtual Reality B1 total = 0
0
Virtual Reality B2 total = 0
0
Virtual Reality B3 total = 0
0
Virtual Reality B4 total = 0
0
Marketing Management B1 total = 50
0
Marketing Management B2 total = 50
0
Marketing Management B3 total = 50
0
Marketing Management B4 total = 50
0
Health Science B1 total = 50
0
Health Science B2 total = 50
0
Health Science B3 total = 50
0
Health Science B4 total = 50
0
Audio Visual Production B1 total = 50
0
Audio Visual Production B2 total = 50
0
Audio Visual Production B3 total = 50
0
Audio Visual Production B4 total = 50
0
Veterinary & Animal Science/Horticulture B1 total = 50
0
Veterinary & Animal Science/Horticulture B2 total = 50
0
Veterinary & Animal Science/Horticulture B3 total = 50
0
Veterinary & Animal Science/Horticulture B4 total = 50
0
Educational Counseling & Social Services B1 total = 50
0
Educational Counseling & Social Services B2 total = 50
0
Educational Counseling & Social Services B3 total = 50
0
Educational Counseling & Social Services B4 t

In [30]:
list(all_students_df.B1.unique())

['Cosmetology/Barbering',
 'Leadership in Government',
 'Networking Systems',
 'Criminal Justice & Correction Services',
 'Marketing Management, A',
 'Virtual Reality, A',
 'Health Science, A',
 'Veterinary & Animal Science/Horticulture, A',
 'Audio Visual Production, A',
 'Educational Counseling & Social Services, A',
 'Virtual Reality, B',
 'Marketing Management, B',
 'Educational Counseling & Social Services, B',
 'Health Science, B',
 'Audio Visual Production, B',
 'Veterinary & Animal Science/Horticulture, B']

### Overwrite Original Files

In [39]:
# dependencies for export_POS_rosters()
pos_str_exceptions, event_year, school = set(), 2023, schools[0]

# overwrite room assignments
room_assignments_df.to_csv(ra_p)

# overwrite all students file
all_students_df.to_csv(as_p)

# write POS rosters (will not get rid of original POS rosters that now have split, A/B)
export_POS_rosters(pos_keys=list(all_students_df.B1.unique()), cleaned_df=all_students_df)